In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel('Prepared_Dataset.xlsx')

In [3]:
df.head()

,S.No,Questions,Modal Answer (ChatGPT),User Answer (Web),Actual Score (0-10),Final Score (Calculated)
0,1,List the advantages of DBMS?,1. Data Integrity: DBMS ensures data accuracy ...,1.Better Data Transferring: Database managemen...,7,NaN
1,2,List the database Applications?,1. MySQL 2. Oracle Database 3. Microsoft SQL S...,1.Oracle Database 2.PostgreSQL 3.SQLite 4.File...,5,NaN
2,3,Define instances and schemas of database?,Instance: An instance is a copy of the databas...,The overall design of the database is called d...,4,NaN
3,4,Discuss Data Independence?,Data independence is the ability of a system t...,Data independence is the ability to modify the...,7,NaN
4,5,Explain database Access for applications Programs,Database Access for applications Programs is a...,Database Access means functionality required i...,0,NaN


In [4]:
mod_ans_set = df.iloc[:, 2].values
usr_ans_set = df.iloc[:, 3].values

In [5]:
type(mod_ans_set)

numpy.ndarray

In [6]:
type(usr_ans_set)

numpy.ndarray

In [7]:
number_of_inputs = len(mod_ans_set)
sim_score = [0]*number_of_inputs
sbert_score = [0]*number_of_inputs

<h1 style="text-align: center">Cosine Similarity</h1>

In [8]:
from sklearn.metrics.pairwise import cosine_similarity
from numpy.linalg import norm

def cos_sim(sent1_emb, sent2_emb):
  cos = np.dot(sent1_emb, sent2_emb)/(norm(sent1_emb)*norm(sent2_emb))
  return cos

<h1 style="text-align: center">Similarity between two texts</h1>

In [9]:
from nltk.tokenize import sent_tokenize
from sentence_transformers import SentenceTransformer
from sentence_transformers import CrossEncoder

def similarity_score(text1, text2, model):
  mod_sent = sent_tokenize(text1)
  usr_sent = sent_tokenize(text2)

  mod_emb = []
  usr_emb = []

  for sent in mod_sent:
    sent_emb = model.encode(sent)
    mod_emb.append(sent_emb)
  
  for sent in usr_sent:
    sent_emb = model.encode(sent)
    usr_emb.append(sent_emb)

  n = len(mod_sent)
  m = len(usr_sent)

  sim_ans = 0

  for i in range(0, n):
    for j in range(0, m):
      if(cos_sim(mod_emb[i], usr_emb[j]) >= 0.7):
        sim_ans += 1
        break
  
  sim_ans /= n
  return sim_ans


<h1 style="text-align:center;">Summary of a text</h1>

In [10]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')
device = torch.device('cpu')

def summarize(text):
  preprocessed_text = text.strip().replace('\n', '')
  t5_input_text = 'summarize: ' + preprocessed_text

  tokenized_text = tokenizer.encode(t5_input_text, return_tensors='pt', max_length=512).to(device)

  summary_ids = model.generate(tokenized_text, min_length=30, max_length=120)
  summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

  return summary

c:\Users\harsh\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\models\t5\tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


<h1 style="text-align: center">Generating Similarity Array</h1>

In [11]:
model_simcse = SentenceTransformer('princeton-nlp/sup-simcse-roberta-large')
model_crossencoder = CrossEncoder('cross-encoder/stsb-roberta-base')
model_sbert = SentenceTransformer('stsb-mpnet-base-v2')

No sentence-transformers model found with name C:\Users\harsh/.cache\torch\sentence_transformers\princeton-nlp_sup-simcse-roberta-large. Creating a new one with MEAN pooling.


In [12]:
for i in range(0, number_of_inputs):
  mod_ans = mod_ans_set[i]
  usr_ans = usr_ans_set[i]

  usr_ans += summarize(usr_ans)

  sim_val = similarity_score(mod_ans, usr_ans, model_simcse)

  sim_score[i] = sim_val

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [13]:
simcse_score = sim_score

In [14]:
for i in range(0, number_of_inputs):
  simcse_score[i] = sim_score[i] * 10

In [15]:
type(simcse_score)

list

In [16]:
simcse_score = np.array(simcse_score)

In [17]:
type(simcse_score)

numpy.ndarray

In [18]:
df['Sim_CSE Score'] = simcse_score.tolist()

In [19]:
df.head()

,S.No,Questions,Modal Answer (ChatGPT),User Answer (Web),Actual Score (0-10),Final Score (Calculated),Sim_CSE Score
0,1,List the advantages of DBMS?,1. Data Integrity: DBMS ensures data accuracy ...,1.Better Data Transferring: Database managemen...,7,NaN,6.666667
1,2,List the database Applications?,1. MySQL 2. Oracle Database 3. Microsoft SQL S...,1.Oracle Database 2.PostgreSQL 3.SQLite 4.File...,5,NaN,1.818182
2,3,Define instances and schemas of database?,Instance: An instance is a copy of the databas...,The overall design of the database is called d...,4,NaN,10.000000
3,4,Discuss Data Independence?,Data independence is the ability of a system t...,Data independence is the ability to modify the...,7,NaN,6.666667
4,5,Explain database Access for applications Programs,Database Access for applications Programs is a...,Database Access means functionality required i...,0,NaN,8.000000


In [20]:
df.to_excel("Prepared_Dataset_Scored.xlsx")

In [21]:
# Testing with random texts
text1 = mod_ans_set[2]
text2 = usr_ans_set[2]

text2 += summarize(text2)

In [22]:
text1

'Instance: An instance is a copy of the database that resides in memory. It contains all the data stored in the database and is used to manipulate that data.  Schema: A schema is a blueprint or map of the database which defines its structure, tables, fields, relationships, views and other elements. It serves as a guide for creating and maintaining the integrity of the database.'

In [23]:
text2

'The overall design of the database is called database schema. Schema will not be changed frequently. It is the logical structure of a database. It does not show the data in the database.Instance or extension or database state is a collection of information that stored in a database at a particular moment is called an instance of the database. The Database instance refers to the information stored in the database at a given point of time. Thus, it is a dynamic value which keeps on changing.schema will not be changed frequently. it is the logical structure of a database. it does not show the data in the database.'

In [24]:
similarity_score(text1, text2, model_simcse)

1.0

In [25]:
for i in range(0, number_of_inputs):
  mod_ans = mod_ans_set[i]
  usr_ans = usr_ans_set[i]

  usr_ans += summarize(usr_ans)

  sbert_val = similarity_score(mod_ans, usr_ans, model_sbert)

  sbert_score[i] = sbert_val

In [28]:
sbert_scoring = sbert_score

for i in range(0, number_of_inputs):
    sbert_scoring[i] = sbert_scoring[i]*10

print(sbert_scoring)

[1.6666666666666665, 0.0, 7.5, 6.666666666666666, 2.0, 5.0, 10.0, 0.0, 3.333333333333333, 2.5, 5.0, 10.0, 6.153846153846154, 8.0, 0.0, 6.0, 1.25, 6.0, 6.363636363636363, 7.0, 1.25, 6.0, 10.0, 0.0, 4.285714285714286, 10.0, 5.0, 0.0, 10.0, 2.5, 6.0, 6.666666666666666, 5.0, 6.666666666666666, 3.333333333333333, 7.5, 10.0, 8.88888888888889, 6.666666666666666, 5.0, 6.0, 10.0, 10.0, 4.444444444444445, 0.0, 6.0, 10.0, 10.0, 10.0, 0.0]


In [31]:
sbert_scoring = np.array(sbert_scoring)

In [32]:
df['SBERT Score'] = sbert_scoring.tolist()

In [33]:
df.head()

,S.No,Questions,Modal Answer (ChatGPT),User Answer (Web),Actual Score (0-10),Final Score (Calculated),Sim_CSE Score,SBERT Score
0,1,List the advantages of DBMS?,1. Data Integrity: DBMS ensures data accuracy ...,1.Better Data Transferring: Database managemen...,7,NaN,6.666667,1.666667
1,2,List the database Applications?,1. MySQL 2. Oracle Database 3. Microsoft SQL S...,1.Oracle Database 2.PostgreSQL 3.SQLite 4.File...,5,NaN,1.818182,0.000000
2,3,Define instances and schemas of database?,Instance: An instance is a copy of the databas...,The overall design of the database is called d...,4,NaN,10.000000,7.500000
3,4,Discuss Data Independence?,Data independence is the ability of a system t...,Data independence is the ability to modify the...,7,NaN,6.666667,6.666667
4,5,Explain database Access for applications Programs,Database Access for applications Programs is a...,Database Access means functionality required i...,0,NaN,8.000000,2.000000


In [34]:
df.to_excel("Prepared_Dataset_Scored.xlsx")